In [ ]:
import pandas as pd
import json

import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

db = sqlalchemy.engine.url.URL(drivername='mysql',
                            host='***********',
                            database='Audio_features',
                            username = "************", password = "************", port=None,
                            query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine = sqlalchemy.create_engine(db)

In [2]:
table = 'feat$cat_ZClaatch_100_cp_w$merged_data$message_id$1gra'
msg_table = 'merged_data'

In [3]:
# read tables
tbl = pd.read_sql_table(table, engine)
msg_tbl = pd.read_sql_table(msg_table, engine)
msg_tbl = msg_tbl[['message_id', 'message', 'KlaatchID_x', 'Date_x' , 'CEL_Total', 'CELVAL1', 'CELVAL2', 'CELVAL3']]

In [4]:
top_10_words_by_topic = pd.read_csv('/home/karthik9/karthik_klutch/50TOPICS.csv')

In [5]:
# drop row where feat == '_intercept'
tbl = tbl[tbl['feat'] != '_intercept']
# convert feat to int
tbl['feat'] = tbl['feat'].astype(int)

tbl = tbl.rename(columns={'feat': 'category'})


In [6]:
tbl = tbl.merge(msg_tbl, left_on='group_id', right_on='message_id', how='left')

In [7]:
# subset tbl to where message > 5 words
tbl_5_words = tbl[tbl['message'].str.split().str.len() > 5]

In [8]:
top_10_msgs = tbl.groupby('category').apply(lambda x: x.nlargest(10, 'group_norm')).reset_index(drop=True)


In [9]:
# merge top 3 msgs with top 10 words
top_10_msgs = top_10_msgs.merge(top_10_words_by_topic, on='category', how='left')
K=top_10_msgs[['category', 'message_id', 'message','top_10_words']]
K

,category,message_id,message,top_10_words
0,0,774_2023-07-25,"Hello? Yes. Oh, hi, Karen. How are you doing? ...","group, meeting, fun, groups, exactly, pizza, s..."
1,0,763_2023-07-31,"Hello. Good. How are you? Yes, I am. The. So f...","group, meeting, fun, groups, exactly, pizza, s..."
2,0,774_2024-03-22,"ello is. Oh. Hi, Karen. How are you doing? Oh,...","group, meeting, fun, groups, exactly, pizza, s..."
3,0,774_2023-03-09,[00:00:06] Reports claim to show that the FTC ...,"group, meeting, fun, groups, exactly, pizza, s..."
4,0,774_2022-12-08,"Hello? Hey, Michelle. Good, Good. How are you ...","group, meeting, fun, groups, exactly, pizza, s..."
...,...,...,...,...
995,99,634_2023-05-31,Hello. Huh? Okay. Okay. Keep leaving messages ...,"kind, also, own, actually, pretty, towers, mus..."
996,99,634_2021-05-24,"Hello. I can't. Oh, here, let me put it on spe...","kind, also, own, actually, pretty, towers, mus..."
997,99,634_2022-10-10,"Hello? Yes. Oh, hey, Ken. Okay. No, Sure. Let ...","kind, also, own, actually, pretty, towers, mus..."
998,99,634_2024-03-18,"ello. Okay. Good. Um. Yeah, sure. That's good....","kind, also, own, actually, pretty, towers, mus..."


In [10]:
# Resolve column conflict
if 'top_10_words' in top_10_msgs.columns:
    top_10_msgs = top_10_msgs.drop(columns=['top_10_words'])

# Merge without conflicts
top_10_msgs = top_10_msgs.merge(top_10_words_by_topic, on='category', how='left')

# Select desired columns
result = top_10_msgs[['category', 'message_id', 'message', 'top_10_words']]

# Save to CSV
result.to_csv('z100.csv', index=False)

print("CSV file 'top_10_messages.csv' has been created.")


CSV file 'top_10_messages.csv' has been created.


In [ ]:
!python /home/karthik9/TheDlatk/dlatk/dlatkInterface.py -d Audio_features -t merged_data -c message_id \
-f  'feat$cat_Klaatch_lda_100_cp_w$merged_data$message_id$1gra'  \
    --outcome_table merged_data  --group_freq_thresh 10 \
    --outcomes CEL_Total CELVAL1 CELVAL2 CELVAL3 \
    --nfold_test_regression --model ridgehighcv --fold_column fold